In [57]:
import os
import nd2
import napari
import pickle
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.measure import regionprops_table
from cardiomyocytes_helper_functions import segment_actin_3D,find_fibers_orientation

In [49]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'
im_name = r'092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2'
path_im = os.path.join(path_dir,im_name)

path_vertices = r'D:\data_analysis\2022_Sahana\masks'

In [50]:
# read in image
im = nd2.imread(path_im)
im.shape

(27, 4, 512, 512)

In [51]:
# read in polygons and masks

pkl_path = os.path.join(path_vertices,im_name.replace('.nd2','_polygons.pkl'))
with open(pkl_path, 'rb') as f:
    vertices_polygons = pickle.load(f)

mask_path = os.path.join(path_vertices,im_name.replace('.nd2','_mask.png'))
mask = imread(mask_path)

In [52]:
# actin segmentation

image_actin = im[:,0,:,:]

image_actin_mask = segment_actin_3D(image_actin)

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [58]:
# flatten segmented actin

image_actin_mask_2D = np.max(image_actin_mask,axis=0)
#theta = find_fibers_orientation(image_actin_mask_2D)

In [53]:
viewer = napari.Viewer()
viewer.add_image(im[:,0,:,:],blending='additive',colormap='magenta')
viewer.add_image(image_actin_mask,blending='additive',colormap='green')

Assistant skips harvesting pyclesperanto as it's not installed.


<Image layer 'image_actin_mask' at 0x25e996f1b80>

In [56]:
viewer.add_image(image_actin_mask_2D,blending='additive',colormap='gray')

<Image layer 'image_actin_mask_2D' at 0x25ea513a2b0>

In [59]:
# calculate properties of cells

properties = ['label','area','centroid','bbox','eccentricity','orientation','intensity_image','image']

im_flat_all_channels = np.max(im,axis=0)
im_flat_all = np.append(im_flat_all_channels,np.expand_dims(image_actin_mask_2D,axis=0),axis=0)
im_flat_all = np.moveaxis(im_flat_all,0,2)

cell_measure = regionprops_table(mask, intensity_image = im_flat_all, properties = properties)
df = pd.DataFrame(cell_measure)
df['image_name'] = im_name

In [60]:
df

,label,area,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,orientation,intensity_image,image,image_name
0,1,28323,415.040815,286.739646,339,134,511,441,0.915018,-1.247227,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
1,2,6917,214.152378,129.314009,141,91,285,165,0.888931,0.147835,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
2,3,18062,222.595615,230.443306,137,148,288,335,0.743125,1.280364,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...


In [66]:

viewer= napari.Viewer()
viewer.add_image(np.moveaxis(df.loc[0,'intensity_image'],2,0))

<Image layer 'Image' at 0x25eb52d59d0>

In [65]:
df.loc[0,'intensity_image'].shape

(172, 307, 5)

In [46]:
# characterize all cells

for i,cell in df.iterrows():

    im_single_cell_actin = cell.intensity_image[:,:,0]

    actin_orientation = find_fibers_orientation(im_single_cell_actin)

    df.loc[i,'actin_orientation'] = actin_orientation



In [47]:
df

,label,area,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,orientation,intensity_image,image,image_name,actin_orientation
0,1,24151,413.470664,302.489089,342,179,512,447,0.895311,-1.132747,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,1.055924
1,2,7701,216.205558,127.639787,138,91,283,171,0.893858,0.188903,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,-0.176715
2,3,19808,212.236319,226.516811,128,149,292,331,0.567734,1.099506,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,-0.844303
3,4,19182,330.324992,236.807267,235,123,402,359,0.910533,-1.163371,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,0.798488


In [ ]:
# calculate distribution at the edge

for i,poly in enumerate(vertices_polygons):

    pl_list = []

    green_max = np.max(im[:,2,:,:],axis=0)

    for i in range(len(vertices_polygons[0])-1):
        
        pl = profile_line(green_max,vertices_polygons[0][i,1:],vertices_polygons[0][i+1,1:])
        
        pl_list.extend(pl)